In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from autorag.data.corpus import langchain_documents_to_parquet
import pandas as pd
from llama_index.llms.ollama import Ollama
from autorag.data.qacreation import generate_qa_llama_index, make_single_content_qa
from langchain_community.chat_models import ChatOllama

/soft/systems/jupyterhub/miniconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
df = pd.read_parquet('/home/btrungvo/workspace/HPCBot/autoRAG/data/qa.parquet')
df.head()

,retrieval_gt,qid,query,generation_gt
0,[[c8612820-7943-4efa-8996-c820be8b711f]],03eda216-f2fd-4ba7-a0ab-55f86f7bac0f,How can I establish an SSH tunnel to connect a...,"[To create a tunnel, on your local machine ope..."
1,[[4a6a29d6-9730-4a2b-8e71-0399dc16fb5d]],063b4bc4-370a-4d31-8917-9a8f772bf749,What happens if jobs are submitted to the dema...,[Jobs in the preemptable queue may be preempte...
2,[[77e7ddcc-1ddc-45e7-a0e7-19f1b36f6cae]],370bb87b-f765-4cd8-b251-b2b6fbe1a04a,What are the platforms available for sycl-ls?,[The platforms available for sycl-ls include I...
3,[[68a0732f-2732-4077-bce4-240db949c700]],05b7111f-db15-4f51-b223-2f83d114a3ea,How to prepare matrix data with ROW-major styl...,[The code prepares matrix data with ROW-major ...
4,[[688e9448-6649-426e-bab9-25753b7cf534]],c0d9770f-1bb9-4e2a-a1e2-56e5ebab628f,What is the most frequent system call in the g...,[The most frequent system call in the given te...


In [4]:
df['query'][0] = "How to login to polaris"
df['generation_gt'][0] = "Log in with \"ssh <username>@polaris.alcf.anl.gov\" and enter the password from your CRYPTOCard/MobilePASS+ token."
df.head()

,retrieval_gt,qid,query,generation_gt
0,[[c8612820-7943-4efa-8996-c820be8b711f]],03eda216-f2fd-4ba7-a0ab-55f86f7bac0f,How to login to polaris,"Log in with ""ssh <username>@polaris.alcf.anl.g..."
1,[[4a6a29d6-9730-4a2b-8e71-0399dc16fb5d]],063b4bc4-370a-4d31-8917-9a8f772bf749,What happens if jobs are submitted to the dema...,[Jobs in the preemptable queue may be preempte...
2,[[77e7ddcc-1ddc-45e7-a0e7-19f1b36f6cae]],370bb87b-f765-4cd8-b251-b2b6fbe1a04a,What are the platforms available for sycl-ls?,[The platforms available for sycl-ls include I...
3,[[68a0732f-2732-4077-bce4-240db949c700]],05b7111f-db15-4f51-b223-2f83d114a3ea,How to prepare matrix data with ROW-major styl...,[The code prepares matrix data with ROW-major ...
4,[[688e9448-6649-426e-bab9-25753b7cf534]],c0d9770f-1bb9-4e2a-a1e2-56e5ebab628f,What is the most frequent system call in the g...,[The most frequent system call in the given te...


In [23]:
i = 10
df['query'][i] = "What is the default programming environment on Polaris?"
df['generation_gt'][i] = "The default programming environment on Polaris is PrgEnv-nvhpc"
df.head()

,retrieval_gt,qid,query,generation_gt
0,[[c8612820-7943-4efa-8996-c820be8b711f]],03eda216-f2fd-4ba7-a0ab-55f86f7bac0f,How to login to polaris,"Log in with ""ssh <username>@polaris.alcf.anl.g..."
1,[[4a6a29d6-9730-4a2b-8e71-0399dc16fb5d]],063b4bc4-370a-4d31-8917-9a8f772bf749,How can I run a job on Polaris? Write an examp...,To run a job on Polaris and submit an example ...
2,[[77e7ddcc-1ddc-45e7-a0e7-19f1b36f6cae]],370bb87b-f765-4cd8-b251-b2b6fbe1a04a,Do you use spack to build modules?,Some modules are generated with spack but not ...
3,[[68a0732f-2732-4077-bce4-240db949c700]],05b7111f-db15-4f51-b223-2f83d114a3ea,What does the (D) mean next to the module names?,(D) means default
4,[[688e9448-6649-426e-bab9-25753b7cf534]],c0d9770f-1bb9-4e2a-a1e2-56e5ebab628f,What is Gromacs,GROMACS is a molecular dynamics package used t...


In [16]:
df = df.drop(index = range(13,50))
df

,retrieval_gt,qid,query,generation_gt
0,[[c8612820-7943-4efa-8996-c820be8b711f]],03eda216-f2fd-4ba7-a0ab-55f86f7bac0f,How to login to polaris,"Log in with ""ssh <username>@polaris.alcf.anl.g..."
1,[[4a6a29d6-9730-4a2b-8e71-0399dc16fb5d]],063b4bc4-370a-4d31-8917-9a8f772bf749,How can I run a job on Polaris? Write an examp...,To run a job on Polaris and submit an example ...
2,[[77e7ddcc-1ddc-45e7-a0e7-19f1b36f6cae]],370bb87b-f765-4cd8-b251-b2b6fbe1a04a,Do you use spack to build modules?,Some modules are generated with spack but not ...
3,[[68a0732f-2732-4077-bce4-240db949c700]],05b7111f-db15-4f51-b223-2f83d114a3ea,What does the (D) mean next to the module names?,(D) means default
4,[[688e9448-6649-426e-bab9-25753b7cf534]],c0d9770f-1bb9-4e2a-a1e2-56e5ebab628f,What is the most frequent system call in the g...,[The most frequent system call in the given te...
5,[[bb2f2876-08ff-4887-b95b-d8d6b7e881de]],960a6a84-b1cb-426d-a8a9-a1820cab277d,what is the difference between LCRC and ALCF?,LCRC is for Argonne researchers and collaborat...
6,[[df27f8b9-9b53-4da6-b7dd-8ce54e5b59a3]],c6c47d15-b223-4ca4-9846-fe9399684743,What does E mean in the qstat queue status?,Exiting
7,[[c745f4d1-7915-4cb2-9f5b-e4c394e6d1f2]],8c24d984-ef81-4fed-a7aa-66ebea15013a,What happens if jobs are submitted to the dema...,Jobs in the preemptable queue may be preempted...
8,[[956471c7-2423-475a-a85a-89f393389e35]],dcebcede-73bb-4ac8-9b11-36c504f14fcf,What is required for an application that requi...,The environment variable MPICH_GPU_SUPPORT_ENA...
9,[[148be2ba-f2df-46dd-8873-63979474a9c6]],cbaecd5f-b1bf-497b-956b-e957739c22db,Is that possible to setup a persistent MongoDB...,you need to send an email to support@alcf.anl....


In [24]:
df

,retrieval_gt,qid,query,generation_gt
0,[[c8612820-7943-4efa-8996-c820be8b711f]],03eda216-f2fd-4ba7-a0ab-55f86f7bac0f,How to login to polaris,"Log in with ""ssh <username>@polaris.alcf.anl.g..."
1,[[4a6a29d6-9730-4a2b-8e71-0399dc16fb5d]],063b4bc4-370a-4d31-8917-9a8f772bf749,How can I run a job on Polaris? Write an examp...,To run a job on Polaris and submit an example ...
2,[[77e7ddcc-1ddc-45e7-a0e7-19f1b36f6cae]],370bb87b-f765-4cd8-b251-b2b6fbe1a04a,Do you use spack to build modules?,Some modules are generated with spack but not ...
3,[[68a0732f-2732-4077-bce4-240db949c700]],05b7111f-db15-4f51-b223-2f83d114a3ea,What does the (D) mean next to the module names?,(D) means default
4,[[688e9448-6649-426e-bab9-25753b7cf534]],c0d9770f-1bb9-4e2a-a1e2-56e5ebab628f,What is Gromacs,GROMACS is a molecular dynamics package used t...
5,[[bb2f2876-08ff-4887-b95b-d8d6b7e881de]],960a6a84-b1cb-426d-a8a9-a1820cab277d,what is the difference between LCRC and ALCF?,LCRC is for Argonne researchers and collaborat...
6,[[df27f8b9-9b53-4da6-b7dd-8ce54e5b59a3]],c6c47d15-b223-4ca4-9846-fe9399684743,What does E mean in the qstat queue status?,Exiting
7,[[c745f4d1-7915-4cb2-9f5b-e4c394e6d1f2]],8c24d984-ef81-4fed-a7aa-66ebea15013a,What happens if jobs are submitted to the dema...,Jobs in the preemptable queue may be preempted...
8,[[956471c7-2423-475a-a85a-89f393389e35]],dcebcede-73bb-4ac8-9b11-36c504f14fcf,What is required for an application that requi...,The environment variable MPICH_GPU_SUPPORT_ENA...
9,[[148be2ba-f2df-46dd-8873-63979474a9c6]],cbaecd5f-b1bf-497b-956b-e957739c22db,Is that possible to setup a persistent MongoDB...,you need to send an email to support@alcf.anl....


In [25]:
df.to_parquet("/home/btrungvo/workspace/HPCBot/autoRAG/data/qa_manual.parquet")